## Bootstraping - Scripts in (R, Python and Julia)
For the bootstrap section you have to use the next equation:


log(inuidur1)~T4+ (female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)

No quadratic tem is required!!!
Next, you have to compute the standard errors of 1,000 bootstrap estimates for the T4, female and black variables.
Describe in detail each step you follow, and what lines of code you changed.
Finally, present your results in a table.

Usar la base de datos penn_jae.dat


In [193]:
import Pkg; Pkg.add("FilePaths")
import Pkg; Pkg.add("TypedTables")
import Pkg; Pkg.add("MacroTools")
import Pkg; Pkg.add("DataTables")
import Pkg; Pkg.add("GLM")

   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed ReadOnlyArrays ─ v0.1.1
   Installed DataTables ───── v0.1.0
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [743a1d0a] + DataTables v0.1.0
    Updating `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
  [743a1d0a] + DataTables v0.1.0
  [988b38a3] + ReadOnlyArrays v0.1.1
Precompiling project...
  ✓ ReadOnlyArrays
  ✓ DataTables
  2 dependencies successfully precompiled in 1 seconds (216 already

In [194]:
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, 
DataStructures, DataTables,StatsBase, Statistics, NamedArrays, PrettyTables, StatsModels, Combinatorics, CSV, DelimitedFiles, DataFrames, Lasso, FilePaths, Combinatorics, CategoricalArrays, TypedTables, MacroTools, PrettyTables


In [195]:
penn, head = readdlm("../../data/penn_jae.dat", header=true, Float64)
penn
df =DataFrame(penn, vec(head))
describe(df)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Nothing,Nothing,DataType
1,abdt,10693.6,10404.0,10691.0,10880.0,,,Float64
2,tg,2.56889,0.0,2.0,6.0,,,Float64
3,inuidur1,12.9148,1.0,10.0,52.0,,,Float64
4,inuidur2,12.1938,0.0,9.0,52.0,,,Float64
5,female,0.402142,0.0,0.0,1.0,,,Float64
6,black,0.116653,0.0,0.0,1.0,,,Float64
7,hispanic,0.0363689,0.0,0.0,1.0,,,Float64
8,othrace,0.00575002,0.0,0.0,1.0,,,Float64
9,dep,0.444045,0.0,0.0,2.0,,,Float64


Filter the data to obtain the control group and the treatment group number 4

In [204]:
penn = filter(row -> row[:tg] in [4,0], df)

first(penn,20)


,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10824.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,2.0
2,10824.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10747.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0
4,10607.0,4.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0
5,10831.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,1.0
6,10845.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,0.0
7,10831.0,0.0,9.0,9.0,1.0,0.0,0.0,0.0,1.0
8,10859.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,1.0
9,10516.0,0.0,15.0,15.0,1.0,0.0,0.0,0.0,0.0


In [205]:
# Treatment group n°4
replace!(penn.tg, 4 => 1)


rename!(penn, "tg" => "T4")





penn

,abdt,T4,inuidur1,inuidur2,female,black,hispanic,othrace,dep
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10824.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,2.0
2,10824.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10747.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0
4,10607.0,1.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0
5,10831.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,1.0
6,10845.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,0.0
7,10831.0,0.0,9.0,9.0,1.0,0.0,0.0,0.0,1.0
8,10859.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,1.0
9,10516.0,0.0,15.0,15.0,1.0,0.0,0.0,0.0,0.0


In [206]:
n = size(penn)[1]

5099

Bootstrap Function

In [217]:
function boot_fn(data,index)
    model = lm(@formula(log(inuidur1)~T4+ (female+black+othrace+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)), penn[index,:])
    T4 = GLM.coeftable(model).cols[1][2]
    female = GLM.coeftable(model).cols[1][3]
    black = GLM.coeftable(model).cols[1][4]
    return [T4, female, black]
end


boot_fn (generic function with 1 method)

In [219]:
boot_fn(penn,[1:n;])

3-element Vector{Float64}:
 -0.07109653401371768
  0.12173628809565984
 -0.2978610361314576

Computing the standard errors of 1,000 bootstrap estimates for the T4, female and black variables.

In [220]:
function boot_2(data,func,R)
            T4_boot2 = []
            fem_boot2 = []
            black_boot2 = []
            for i in 1:R
                append!(T4_boot2,func(data,sample([1:5099;], n, replace = true))[1])
                append!(fem_boot2,func(data,sample([1:5099;], n, replace = true))[2])
                append!(black_boot2,func(data,sample([1:5099;], n, replace = true))[3])
            end
        table = NamedArray(zeros(3, 3))

        table[1,2] = mean(T4_boot2)
        table[1,3] = std(T4_boot2, corrected=true)
        table[2,2] = mean(fem_boot2)
        table[2,3] = std(fem_boot2, corrected=true)
        table[3,2] = mean(black_boot2)
        table[3,3] = std(black_boot2, corrected=true)
    
        T = DataFrame(table, [ :"Variable", :"Coefficient (boostrap)", :"Standar error (boostrap)"]) 
        T[!,:Variable] = string.(T[!,:Variable]) 

        T[1,1] = "T4_boot2"
        T[2,1] = "fem_boot2"
        T[3,1] = "black_boot2"
        
        bootstrap_statistics = Dict{String,Any}("Table" => T, "T4_boot2" => T4_boot2, "fem_boot2" => fem_boot2,
        "black_boot2" => black_boot2)
    return bootstrap_statistics
end

boot_2 (generic function with 1 method)

In [222]:
boot_2(Auto,boot_fn,1000)["Table"]

,Variable,Coefficient (boostrap),Standar error (boostrap)
,String,Float64,Float64
1,T4_boot2,-0.0726805,0.0372745
2,fem_boot2,0.122096,0.0341478
3,black_boot2,-0.29879,0.0592103


## Comparative models (R, Python and Julia)

In [47]:

import Pkg; Pkg.add("CodecBzip2")
import Pkg; Pkg.add("StatsBase")

    Updating registry at `C:\Users\acoap\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [2913bbd2] + StatsBase v0.33.16
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`


In [49]:
using RData, LinearAlgebra, GLM, DataFrames, Statistics, Random, Distributions, DataStructures, NamedArrays, PrettyTables,
        Plots, StatsBase,StatsPlots, GLM, CodecBzip2

In [27]:
rdata_read = load("../../data/cps2012.RData")
data = rdata_read["data"]

,year,lnw,female,widowed,divorced,separated,nevermarried,hsd08
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2012.0,1.90954,1.0,0.0,0.0,0.0,0.0,0.0
2,2012.0,1.36577,1.0,0.0,0.0,0.0,0.0,0.0
3,2012.0,2.54022,0.0,0.0,0.0,0.0,0.0,0.0
4,2012.0,1.80109,1.0,0.0,0.0,0.0,0.0,0.0
5,2012.0,3.3499,0.0,0.0,0.0,0.0,0.0,0.0
6,2012.0,2.00283,0.0,0.0,0.0,0.0,0.0,0.0
7,2012.0,2.45609,0.0,0.0,0.0,0.0,1.0,0.0
8,2012.0,3.57305,0.0,0.0,0.0,0.0,0.0,0.0
9,2012.0,2.51366,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
Z = select( data, Not( ["year", "lnw"]) )
names( Z )

21-element Vector{String}:
 "female"
 "widowed"
 "divorced"
 "separated"
 "nevermarried"
 "hsd08"
 "hsd911"
 "hsg"
 "cg"
 "ad"
 "mw"
 "so"
 "we"
 "exp1"
 "exp2"
 "exp3"
 "exp4"
 "weight"
 "married"
 "ne"
 "sc"

In [223]:
#Splitting the data
Random.seed!(1234)

TaskLocalRNG()

In [224]:
training = sample( collect(1:nrow( data ) ), trunc(Int, 3 * nrow( data ) / 4 ),  replace= false )

data_train = data[ vec(training), : ]
data_test = data[ Not(training), : ]

,year,lnw,female,widowed,divorced,separated,nevermarried,hsd08
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2012.0,1.90954,1.0,0.0,0.0,0.0,0.0,0.0
2,2012.0,2.00283,0.0,0.0,0.0,0.0,0.0,0.0
3,2012.0,0.289633,1.0,0.0,0.0,0.0,0.0,0.0
4,2012.0,3.14226,1.0,0.0,1.0,0.0,0.0,0.0
5,2012.0,1.75389,0.0,0.0,0.0,0.0,1.0,0.0
6,2012.0,2.05892,0.0,0.0,0.0,0.0,0.0,0.0
7,2012.0,2.42792,0.0,0.0,0.0,0.0,0.0,0.0
8,2012.0,3.37343,0.0,0.0,0.0,0.0,0.0,0.0
9,2012.0,2.52323,1.0,0.0,0.0,0.0,0.0,0.0


We construct the two different model matrices  𝑋𝑏𝑎𝑠𝑖𝑐  and  𝑋𝑓𝑙𝑒𝑥  for both the training and the test sample:

In [225]:
size(data_test)

(7305, 23)

In [226]:
size(data_train)

(21912, 23)

In [34]:
X_basic =  "female + female : (widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3)"
X_flex = "female + female : (widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) + (widowed + divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so +we + exp1 + exp2 + exp3) ^ 2"

"female + female : (widowed + divorced + separated + nevermarried +\nhsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) + (widowed + divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so +we + exp1 + exp2 + exp3) ^ 2"

In [227]:
formula_basic = @formula(lnw ~ (female + female + (widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3)))

FormulaTerm
Response:
  lnw(unknown)
Predictors:
  female(unknown)
  widowed(unknown)
  divorced(unknown)
  separated(unknown)
  nevermarried(unknown)
  hsd08(unknown)
  hsd911(unknown)
  hsg(unknown)
  cg(unknown)
  ad(unknown)
  mw(unknown)
  so(unknown)
  we(unknown)
  exp1(unknown)
  exp2(unknown)
  exp3(unknown)

In [228]:
formula_flex = @formula(lnw ~ (female + female +(widowed + divorced + separated + nevermarried +
hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) + (widowed + divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so +we + exp1 + exp2 + exp3) ^ 2))


FormulaTerm
Response:
  lnw(unknown)
Predictors:
  female(unknown)
  widowed(unknown)
  divorced(unknown)
  separated(unknown)
  nevermarried(unknown)
  hsd08(unknown)
  hsd911(unknown)
  hsg(unknown)
  cg(unknown)
  ad(unknown)
  mw(unknown)
  so(unknown)
  we(unknown)
  exp1(unknown)
  exp2(unknown)
  exp3(unknown)
  (widowed,divorced,separated,nevermarried,hsd08,hsd911,hsg,cg,ad,mw,so,we,exp1,exp2,exp3)->(widowed + divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3) ^ 2

In [229]:
model_X_basic_train = ModelMatrix(ModelFrame(formula_basic,data_train)).m
model_X_basic_test = ModelMatrix(ModelFrame(formula_basic,data_test)).m
p_basic = size(model_X_basic_test)[2]

17

In [230]:
model_X_flex_train = ModelMatrix(ModelFrame(formula_flex,data_train)).m
model_X_flex_test = ModelMatrix(ModelFrame(formula_flex,data_test)).m
p_flex = size(model_X_flex_test)[2]

18

In [134]:
Y_train = data_train[!, ["lnw"]] # Dataframe format
Y_test = data_test[ !,  ["lnw"]]

,lnw
,Float64
1,1.90954
2,2.00283
3,0.289633
4,3.14226
5,1.75389
6,2.05892
7,2.42792
8,3.37343
9,2.52323


In [89]:
p_basic
p_flex

18

# OLS


We fit the basic model to our training data by running an ols regression and compute the mean squared error on the test sample.

In [90]:
data_train

,year,lnw,female,widowed,divorced,separated,nevermarried,hsd08
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2012.0,3.2164,1.0,0.0,0.0,0.0,0.0,0.0
2,2012.0,3.46769,0.0,0.0,0.0,0.0,0.0,0.0
3,2012.0,2.75207,0.0,0.0,0.0,0.0,0.0,0.0
4,2012.0,3.38345,1.0,0.0,0.0,0.0,0.0,0.0
5,2012.0,3.18885,1.0,0.0,0.0,0.0,0.0,0.0
6,2012.0,2.46439,1.0,0.0,0.0,0.0,0.0,0.0
7,2012.0,3.22207,0.0,0.0,0.0,0.0,0.0,0.0
8,2012.0,2.26471,1.0,1.0,0.0,0.0,0.0,0.0
9,2012.0,2.99323,1.0,0.0,0.0,0.0,0.0,0.0


In [91]:
size( data_train )

(21912, 23)

In [92]:
fit_lm_basic = lm(formula_basic, data_train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lnw ~ 1 + female + widowed + divorced + separated + nevermarried + hsd08 + hsd911 + hsg + cg + ad + mw + so + we + exp1 + exp2 + exp3

Coefficients:
────────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error       t  Pr(>|t|)    Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────────
(Intercept)    2.43808     0.0297804    81.87    <1e-99   2.37971      2.49646
female        -0.2693      0.00796776  -33.80    <1e-99  -0.284918    -0.253683
widowed       -0.191484    0.0456718    -4.19    <1e-04  -0.281005    -0.101964
divorced      -0.0731377   0.0125568    -5.82    <1e-08  -0.09775     -0.0485254
separated     -0.0949229   0.0309356    -3.07    0.0022  -0.155559    -0.0342869
nevermarried  -0.130873    0.011549    -11.

In [93]:
# Compute the Out-Of-Sample Performance
yhat_lm_basic = GLM.predict( fit_lm_basic , data_test )
res_lm_basic = ( Y_test[!,1] - yhat_lm_basic ).^ 2
print("The mean squared error (MSE) using the basic model is equal to " , mean( res_lm_basic ) ) # MSE OLS (basic model)  

The mean squared error (MSE) using the basic model is equal to 0.33764952933988224

To determine the out-of-sample  𝑀𝑆𝐸  and the standard error in one step, we can use the function lm:

In [94]:
matrix_ones = ones( size(res_lm_basic)[1] ,1 )
mean_residuals = lm(  matrix_ones, res_lm_basic )   # first argument (X), secind argument (Y)
MSE_lm_basic = [ coef( mean_residuals ) , stderror( mean_residuals ) ]
MSE_lm_basic

2-element Vector{Vector{Float64}}:
 [0.3376495293398821]
 [0.02338038641025037]

We also compute the out-of-sample  𝑅2 :

In [95]:
R2_lm_basic = 1 .- ( MSE_lm_basic[1] / var( Y_test[!,1] ) )
print( "The R^2 using the basic model is equal to ", R2_lm_basic[1] ) # MSE OLS (basic model) 

The R^2 using the basic model is equal to 0.2469955679371647

We repeat the same procedure for the flexible model.

In [107]:
# ols (flexible model)
fit_lm_flex = lm( formula_flex, data_train ) 
yhat_lm_flex = GLM.predict( fit_lm_flex, data_test)

res_lm_flex = ( Y_test[!,1] - yhat_lm_flex ) .^ 2
mean_residuals = lm(  matrix_ones, res_lm_flex )
MSE_lm_flex = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

R2_lm_flex = 1 .- ( MSE_lm_flex[1] / var( Y_test[!,1] ) )
print( "The R^2 using the basic model is equal to ", R2_lm_flex[1] ) # MSE OLS (flex model) 

The R^2 using the basic model is equal to 0.24698185407019946

## Lasso, Ridge and Elastic Net

Considering the basic model, we run a lasso/post-lasso regression first and then we compute the measures for the out-of-sample performance. Note that applying the package hdm and the function rlasso we rely on a theoretical based choice of the penalty level  𝜆  in the lasso regression.

In [135]:

import Pkg;Pkg.add("FreqTables")

import Pkg; Pkg.add("TableOperations")
using DelimitedFiles, DataFrames, Lasso

   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`


In [136]:
# load HDM package

include("C:\\Users\\acoap\\Documents\\Github_\\ECO224\\Labs\\Julia_Notebooks\\hdmjl\\hdmjl.jl")

In [137]:
names_col1 = Symbol.(coefnames(fit_lm_basic))
X1 = DataFrame(model_X_basic_train, names_col1 )

,(Intercept),female,widowed,divorced,separated,nevermarried,hsd08,hsd911
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [231]:
# basic model 
# not post - lasso (HDM)
# first false for Not post lasso, second false for not intercetp


rlasso_basic  = rlasso_arg( X1, Y_train, nothing, false, false, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

fit_rlasso_basic = rlasso(rlasso_basic)


# post - lasso (HDM)
rlasso_basic_post  = rlasso_arg( X1, Y_train, nothing, true, false, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

fit_rlasso_basic_post = rlasso(rlasso_basic_post)

yhat_rlasso = model_X_basic_test*fit_rlasso_basic["coefficients"] 
yhat_rlasso_post = model_X_basic_test*fit_rlasso_basic_post["coefficients"] 


res_rlasso_basic = ( Y_test[!,1] - yhat_rlasso ).^ 2
matrix_ones = ones( size(res_rlasso_basic)[1] ,1 )
mean_residuals = lm(  matrix_ones, res_rlasso_basic )  
MSE_rlasso_basic = [ coef( mean_residuals ) , stderror( mean_residuals ) ]
R2_rlasso_basic = 1 .- ( MSE_rlasso_basic[1] / var(Y_test[!,1]) ) 

res_rlasso_basic_post = ( Y_test[!,1] - yhat_rlasso_post ).^ 2
matrix_ones = ones( size(res_rlasso_basic_post)[1] ,1 )
mean_residuals = lm(  matrix_ones, res_rlasso_basic_post )  
MSE_rlasso_basic_post = [ coef( mean_residuals ) , stderror( mean_residuals ) ]
R2_rlasso_basic_post = 1 .- ( MSE_rlasso_basic_post[1] / var(Y_test[!,1]) ) 

LoadError: MethodError: no method matching DataFrame(::Vector{Vector}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\acoap\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\acoap\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\acoap\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

## Flexible Model

In [232]:
names_col2 = Symbol.(coefnames(fit_lm_flex))
X2 = DataFrame(model_X_flex_train, names_col2 )

,(Intercept),female,widowed,divorced,separated,nevermarried,hsd08,hsd911
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [237]:
# Flex - model
# Not post - lasso (HDM)


rlasso_flex  = rlasso_arg( X2, Y_train, nothing, false, false, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

fit_rlasso_flex = rlasso(rlasso_flex)


# post - lasso (HDM)
rlasso_flex_post  = rlasso_arg( X2, Y_train, nothing, true, false, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

fit_rlasso_flex_post = rlasso(rlasso_flex_post)

yhat_rlasso_flex = model_X_flex_test*fit_rlasso_flex["coefficients"] 
yhat_rlasso_flex_post = model_X_flex_test*fit_rlasso_flex_post["coefficients"] 


res_rlasso_flex = ( Y_test[!,1] - yhat_rlasso_flex ).^ 2
matrix_ones = ones( size(res_rlasso_flex)[1] ,1 )
mean_residuals = lm(  matrix_ones, res_rlasso_flex )  
MSE_rlasso_flex = [ coef( mean_residuals ) , stderror( mean_residuals ) ]
R2_rlasso_flex = 1 .- ( MSE_rlasso_flex[1] / var(Y_test[!,1]) ) 

res_rlasso_flex_post = ( Y_test[!,1] - yhat_rlasso_flex_post ).^ 2
matrix_ones = ones( size(res_rlasso_flex_post)[1] ,1 )
mean_residuals = lm(  matrix_ones, res_rlasso_flex_post )  
MSE_rlasso_flex_post = [ coef( mean_residuals ) , stderror( mean_residuals ) ]
R2_rlasso_flex_post = 1 .- ( MSE_rlasso_flex_post[1] / var(Y_test[!,1]) ) 


LoadError: MethodError: no method matching DataFrame(::Vector{Vector}, ::Symbol)
[0mClosest candidates are:
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\acoap\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:249
[0m  DataFrame(::AbstractVector{<:AbstractVector}, [91m::AbstractVector{Symbol}[39m; makeunique, copycols) at C:\Users\acoap\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:242
[0m  DataFrame(::AbstractVector, [91m::AbstractVector{<:AbstractString}[39m; makeunique, copycols) at C:\Users\acoap\.julia\packages\DataFrames\GtZ1l\src\dataframe\dataframe.jl:237
[0m  ...

In [234]:
print( "The R^2 using the basic model is  ", R2_rlasso_flex[1] ," for lasso and ", R2_rlasso_flex_post[1] , " for Post - lasso")

LoadError: UndefVarError: R2_rlasso_flex not defined

Now, we repeat the same procedure for the flexible model.

In [236]:
import Pkg; Pkg.add("GLMNet")
using GLMNet

   Resolving package versions...
   Installed glmnet_jll ─ v5.0.0+0
   Installed GLMNet ───── v0.7.0
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [8d5ece8b] + GLMNet v0.7.0
    Updating `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
  [8d5ece8b] + GLMNet v0.7.0
  [78c6b45d] + glmnet_jll v5.0.0+0
Precompiling project...
  ✓ glmnet_jll
  ✓ GLMNet
  2 dependencies successfully precompiled in 3 seconds (218 already precompiled, 42 skipped during auto due to previous errors)


In [239]:
fit_lasso_cv   = GLMNet.glmnetcv(model_X_basic_train, Y_train[!,1], alpha=1)
fit_ridge   = GLMNet.glmnetcv(model_X_basic_train, Y_train[!,1], alpha=0)
fit_elnet   = GLMNet.glmnetcv(model_X_basic_train, Y_train[!,1], alpha= 0.5)

yhat_lasso_cv    = GLMNet.predict(fit_lasso_cv,  model_X_basic_test)
yhat_ridge   = GLMNet.predict(fit_ridge,  model_X_basic_test)
yhat_elnet   = GLMNet.predict(fit_elnet,  model_X_basic_test)

res_lasso_cv = ( Y_test[!,1] - yhat_lasso_cv ) .^ 2
mean_residuals = lm(  matrix_ones, res_lasso_cv )
MSE_lasso_cv = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

res_ridge = ( Y_test[!,1] - yhat_ridge ) .^ 2
mean_residuals = lm(  matrix_ones, res_ridge )
MSE_ridge = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

res_elnet = ( Y_test[!,1] - yhat_elnet ) .^ 2
mean_residuals = lm(  matrix_ones, res_elnet )
MSE_elnet = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

R2_lasso_cv = 1 .- ( MSE_lasso_cv[1] / var( Y_test[!,1] ) )
R2_ridge = 1 .- ( MSE_ridge[1] / var( Y_test[!,1] ) )
R2_elnet = 1 .- ( MSE_elnet[1] / var( Y_test[!,1] ) )

print("R^2 using cross-validation for lasso, ridge and elastic net in the basic model:"
    , R2_lasso_cv[1],"; ",R2_ridge[1]," y \n",R2_elnet[1])

R^2 using cross-validation for lasso, ridge and elastic net in the basic model:0.24545030031563497; 0.24427869819572956 y 
0.24563351325111782

In [240]:
fit_lasso_cv_flex   = GLMNet.glmnetcv(model_X_flex_train, Y_train[!,1], alpha=1)
fit_ridge_flex   = GLMNet.glmnetcv(model_X_flex_train, Y_train[!,1], alpha=0)
fit_elnet_flex   = GLMNet.glmnetcv(model_X_flex_train, Y_train[!,1], alpha= 0.5)

yhat_lasso_cv_flex    = GLMNet.predict(fit_lasso_cv_flex,  model_X_flex_test)
yhat_ridge_flex   = GLMNet.predict(fit_ridge_flex,  model_X_flex_test)
yhat_elnet_flex   = GLMNet.predict(fit_elnet_flex,  model_X_flex_test)

res_lasso_cv_flex = ( Y_test[!,1] - yhat_lasso_cv_flex ) .^ 2
mean_residuals = lm(  matrix_ones, res_lasso_cv_flex )
MSE_lasso_cv_flex = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

res_ridge_flex = ( Y_test[!,1] - yhat_ridge_flex ) .^ 2
mean_residuals = lm(  matrix_ones, res_ridge_flex )
MSE_ridge_flex = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

res_elnet_flex = ( Y_test[!,1] - yhat_elnet_flex ) .^ 2
mean_residuals = lm(  matrix_ones, res_elnet_flex )
MSE_elnet_flex = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

R2_lasso_cv_flex = ( 1 .- ( MSE_lasso_cv_flex[1] / var( Y_test[!,1] ) ) )[1]
R2_ridge_flex = ( 1 .- ( MSE_ridge_flex[1] / var( Y_test[!,1] ) ) )[1]
R2_elnet_flex = ( 1 .- ( MSE_elnet_flex[1] / var( Y_test[!,1] ) ) )[1]

print("R^2 using cross-validation for lasso, ridge and elastic net in the flex model:"
    ,R2_lasso_cv_flex[1],"; ",R2_ridge_flex[1],"y \n",R2_elnet_flex[1])

R^2 using cross-validation for lasso, ridge and elastic net in the flex model:0.24577847427356259; 0.24397002654385325y 
0.24549121661461581

Comparing, we obtain a similar the perfomance between the Lasso regression with cross-validated penalty is quite similar to the performance of lasso using a theoretical based choice of the tuning parameter.

## Non - Linear Models

In [241]:
import Pkg; Pkg.add( "ScikitLearn" )

import Pkg; Pkg.add("DecisionTree")

   Resolving package versions...
   Installed ScikitLearnBase ─ v0.5.0
   Installed ScikitLearn ───── v0.6.4
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [3646fa90] + ScikitLearn v0.6.4
    Updating `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
  [3646fa90] + ScikitLearn v0.6.4
  [6e75b9c4] + ScikitLearnBase v0.5.0
Precompiling project...
  ✓ ScikitLearnBase
  ✓ ScikitLearn
  2 dependencies successfully precompiled in 4 seconds (220 already precompiled, 42 skipped during auto due to previous errors)
   Resolving package versions...
   Installed AbstractTrees ─ v0.3.4
   Installed DecisionTree ── v0.10.12
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [7806a523] + DecisionTree v0.10.12
    Updating `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
  [1520ce14] + AbstractTrees v0.3.4
  [7806a523] + DecisionTree v0.10.12
Precompiling project...
  ✓ AbstractTrees
  ✓ DecisionTree
  2 dependencies successfully precompiled in 1 

In [242]:
using ScikitLearn, DecisionTree

In [243]:
tree = DecisionTreeRegressor(min_purity_increase = 0, min_samples_leaf=1, min_samples_split = 2,rng = 0)

DecisionTreeRegressor
max_depth:                -1
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     nothing

Now we apply pruning to the complex tree above to reduce the depth in each branches of the tree.

In [247]:
# Using prun purity parameter = 0.010

tree1 = DecisionTreeRegressor( min_samples_leaf=1, min_samples_split = 2, rng = 0, min_purity_increase = 0.01)
trees_fit1 =  ScikitLearn.fit!(tree1, model_X_basic_train, Y_train[!,1] )

DecisionTreeRegressor
max_depth:                -1
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.01
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     Decision Tree
Leaves: 9
Depth:  4

In [250]:
using ScikitLearn.GridSearch: GridSearchCV

In [251]:
gridsearch = GridSearchCV(DecisionTreeRegressor(), Dict(:max_depth => 5:2:15 , :min_samples_split => 2:10:300,
                                            :min_purity_increase => 0.001:0.001:0.005))

GridSearchCV
  estimator: DecisionTreeRegressor
  param_grid: Dict{Symbol, AbstractRange}
  scoring: Nothing nothing
  loss_func: Nothing nothing
  score_func: Nothing nothing
  fit_params: Dict{Any, Any}
  n_jobs: Int64 1
  iid: Bool true
  refit: Bool true
  cv: Nothing nothing
  verbose: Int64 0
  error_score: String "raise"
  scorer_: Nothing nothing
  best_params_: Nothing nothing
  best_score_: Nothing nothing
  grid_scores_: Nothing nothing
  best_estimator_: Nothing nothing


In [252]:
ScikitLearn.fit!(gridsearch,  model_X_basic_train, Y_train[!,1])

GridSearchCV
  estimator: DecisionTreeRegressor
  param_grid: Dict{Symbol, AbstractRange}
  scoring: Nothing nothing
  loss_func: Nothing nothing
  score_func: Nothing nothing
  fit_params: Dict{Any, Any}
  n_jobs: Int64 1
  iid: Bool true
  refit: Bool true
  cv: Nothing nothing
  verbose: Int64 0
  error_score: String "raise"
  scorer_: score (function of type typeof(ScikitLearnBase.score))
  best_params_: Dict{Symbol, Any}
  best_score_: Float64 -0.33582800992147227
  grid_scores_: Array{ScikitLearn.Skcore.CVScoreTuple}((900,))
  best_estimator_: DecisionTreeRegressor


In [253]:
tree_prune = DecisionTreeRegressor(max_depth =15, min_samples_leaf=1, min_samples_split = 152, 
                                    min_purity_increase = 0.002 )
tree_prune_tree =  ScikitLearn.fit!(tree_prune, model_X_basic_train, Y_train[!,1] )

DecisionTreeRegressor
max_depth:                15
min_samples_leaf:         1
min_samples_split:        152
min_purity_increase:      0.002
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     Decision Tree
Leaves: 88
Depth:  13

In [258]:
y_hat_pt = ScikitLearn.predict(trees_fit1, model_X_basic_test)

7305-element Vector{Float64}:
 2.4251145449795386
 2.694345434271034
 2.4251145449795386
 2.4251145449795386
 2.694345434271034
 2.777988202043642
 2.694345434271034
 2.694345434271034
 3.053316789762287
 2.4251145449795386
 2.694345434271034
 2.694345434271034
 2.777988202043642
 ⋮
 2.694345434271034
 3.1929101642475786
 2.4251145449795386
 2.777988202043642
 3.429818935730825
 2.4251145449795386
 2.9038068695659414
 3.1929101642475786
 3.053316789762287
 2.694345434271034
 3.053316789762287
 2.8147885457227977

In [259]:
res_tree = ( Y_test[!,1] - y_hat_pt ) .^ 2
mean_residuals = lm(  matrix_ones, res_tree )
MSE_tree = [ coef( mean_residuals ) , stderror( mean_residuals ) ]

R2_tree = ( 1 .- ( MSE_tree[1] / var( Y_test[!,1] ) ) )[1]

print("R^2 using Prune - tree:", R2_tree)

R^2 using Prune - tree:0.21129102476731831

# Results

In [256]:
table = NamedArray(zeros(15, 4))

table[1,2:3] = [MSE_lm_basic[1][1], MSE_lm_basic[2][1]]
table[2,2:3] = [MSE_lm_flex[1][1], MSE_lm_flex[2][1]]
table[3,2:3] = [MSE_rlasso_basic[1][1], MSE_rlasso_basic[2][1]]
table[4,2:3] = [MSE_rlasso_basic_post[1][1], MSE_rlasso_basic_post[2][1]]
table[5,2:3] = [MSE_rlasso_flex[1][1], MSE_rlasso_flex[2][1]]
table[6,2:3] = [MSE_rlasso_flex_post[1][1], MSE_rlasso_flex_post[2][1]]
table[7,2:3] = [MSE_lasso_cv[1][1], MSE_lasso_cv[2][1]]
table[8,2:3] = [MSE_ridge[1][1], MSE_ridge[2][1]]
table[9,2:3] = [MSE_elnet[1][1], MSE_elnet[2][1]]
table[10,2:3] = [MSE_lasso_cv_flex[1][1], MSE_lasso_cv_flex[2][1]]
table[11,2:3] = [MSE_ridge_flex[1][1], MSE_ridge_flex[2][1]]
table[12,2:3] = [MSE_elnet_flex[1][1], MSE_elnet_flex[2][1]]
table[13,2:3] = [MSE_prune_tree[1][1], MSE_prune_tree[2][1]]
table[14,2:3] = [MSE_rf[1][1], MSE_rf[2][1]]
#table[15,2:3] = [0,0]

table[1,4] = R2_lm_basic[1]
table[2,4] = R2_lm_flex[1]
table[3,4] = R2_rlasso_basic[1]
table[4,4] = R2_rlasso_basic_post[1]
table[5,4] = R2_rlasso_flex[1]
table[6,4] = R2_rlasso_flex_post[1]
table[7,4] = R2_lasso_cv[1]
table[8,4] = R2_ridge[1]
table[9,4] = R2_elnet[1]
table[10,4] = R2_lasso_cv_flex[1]
table[11,4] = R2_ridge_flex[1]
table[12,4] = R2_elnet_flex[1]
table[13,4] = R2_prune_tree[1]
table[14,4] = R2_rf[1]
#table[15,4] = 0

T = DataFrame(table, [ :"Model",:"MSE", :"S.E. for MSE", :"R-squared"]) 
T[!,:Model] = string.(T[!,:Model]) 

T[1,1] = "Least Squares (basic)"
T[2,1] = "Least Squares (flexible)"
T[3,1] = "Lasso"
T[4,1] = "Post-Lasso"
T[5,1] = "Lasso (flexible)"
T[6,1] = "Post-Lasso (flexible)"
T[7,1] = "Cross-Validated lasso"
T[8,1] = "Cross-Validated ridge"
T[9,1] = "Cross-Validated elnet"
T[10,1] = "Cross-Validated lasso (flexible)"
T[11,1] = "Cross-Validated ridge (flexible)"
T[12,1] = "Cross-Validated elnet (flexible)"
T[13,1] = "Pruned Tree"
T[14,1] = "Random Forest"
T[15,1] = "Boosted Trees"

header = (["Model", "MSE", "S.E. for MSE", "R-squared"])

pretty_table(T; backend = Val(:html), header = header, formatters=ft_round(4), alignment=:c)

LoadError: UndefVarError: MSE_rlasso_basic not defined

## How to build a tree regresion?

#### Tree-building process

- First of all, the process is a top-down approach because it starts at the top by mentioning which is the predictor that most affects the outcome variable (output) and chooses the cutoff that minimizes the residual sum of squares in each group divided by the cutoff $ \sum_{i \in R_j} (Y_i - \hat{Y_{Rj}})^2 $ swhere the estimated value of Y is the mean of the Y values for the observations in $Rj$.Therefore, it is also called a greedy approach because at each step of the construction process the best split occurs at that particular step without looking ahead.  
- Starting from the first predictor, the predictor space is successively divided, choosing at each step which predictor most affects or determines the result variable. That way, it ultimately generates regions $ R_1, R_2, ... , R_J $, called terminal nodes or leaves, which are the mean of the values of the dependent variable for the observations in that region, given the predictors that are most measure determine Y in that region.
- The goal is to find the regions $ R_1, R_2, ... , R_J $ that minimize the residual sum of squares (RSS):
\begin{align}
\sum_{j=1}^{j} \sum_{i \in R_j} (Y_i - \hat{Y_{Rj}})^2
\end{align}
where $ \hat{Y_{Rj}} $ is the mean response for the training observations within the jth box.

#### Pruning regression trees

The deeper the tree, that is, the more we decompose and divide the observations according to the predictors, the better the approximation to the regression function. However, it increases the estimation noise because there are fewer observations per terminal node to estimate the predicted value of the node. To improve the predictive performance it is necessary to "prune the tree" or cut branches. The pruning regression tree tries to find the correct depth.